In [1]:
import os

In [2]:
%pwd

'c:\\projects\\hr_attrition\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [5]:

from hr_attrition.constants import *
from hr_attrition.utils.common import read_yaml, create_directories, save_json

In [6]:
from hr_attrition.constants import *
from hr_attrition.utils.common import read_yaml, create_directories, save_json
  # Assuming this is where ModelEvaluationConfig is defined

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.model_params
        schema = self.schema.TARGET_COLUMN

        # Ensure the root directory for model evaluation exists
        create_directories([config.root_dir])

        # Create and return the ModelEvaluationConfig object
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_data_path=Path(config.test_data_path),
            model_path=Path(config.model_path),
            all_params=params,
            metric_file_name=Path(config.metric_file_name),
            target_column=schema.name
        )
        return model_evaluation_config
        


In [8]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from hr_attrition.utils.common import save_json
import joblib

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        """
        Evaluate binary classification metrics.
        """
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        f1 = f1_score(actual, pred)
        roc_auc = roc_auc_score(actual, pred)
        return accuracy, precision, recall, f1, roc_auc

    def save_results(self):
        """
        Load test data, make predictions, evaluate metrics, and save results.
        """
        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # Separate features and target
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        # Ensure target column is numerical (if categorical)
        if test_y.dtype == 'object':
            test_y = test_y.map({"Yes": 1, "No": 0})  # Adjust mapping based on your data

        # Make predictions
        predicted_labels = model.predict(test_x)

        # Evaluate metrics
        accuracy, precision, recall, f1, roc_auc = self.eval_metrics(test_y, predicted_labels)

        # Save metrics as JSON
        scores = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "roc_auc": roc_auc
        }
        save_json(path=self.config.metric_file_name, data=scores)

In [10]:



try:
    # Initialize the configuration manager
    config = ConfigurationManager()

    # Get the model evaluation configuration
    model_eval_config = config.get_model_evaluation_config()

    # Initialize the ModelEvaluation class
    model_evaluation = ModelEvaluation(config=model_eval_config)

    # Perform evaluation and save results
    model_evaluation.save_results()

except FileNotFoundError as e:
    # Handle missing files (e.g., test data or model file)
    print(f"File not found: {e}")
    raise e

except Exception as e:
    # Log and re-raise other exceptions
    print(f"An error occurred during model evaluation: {e}")
    raise e

[2025-04-17 18:39:44,098: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-17 18:39:44,102: INFO: common: yaml file: params.yaml loaded successfully]


[2025-04-17 18:39:44,106: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-17 18:39:44,107: INFO: common: created directory at: artifacts]
[2025-04-17 18:39:44,109: INFO: common: created directory at: artifacts/model_evaluation]
[2025-04-17 18:39:44,915: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
